# buy-open-sell-close

On the first trading day of each week, buy on the open, then sell on the close.

In [1]:
import pandas as pd
import datetime
import pinkfish as pf

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now()

Timeseries

In [3]:
# Fetch timeseries, select.
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)

# Add calendar columns.
ts = pf.calendar(ts)

# Finalize timeseries.
ts, start = pf.finalize_timeseries(ts, start)

# Create tradelog and daily balance objects.
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

In [4]:
ts

,open,high,low,close,adj_close,volume,dotw,dotm,doty,month,first_dotw,first_dotm,first_doty,last_dotw,last_dotm,last_doty
date,,,,,,,,,,,,,,,,
2015-01-02,206.38,206.88,204.18,205.43,176.13,121465900,4,2,2,1,False,True,True,True,False,False
2015-01-05,204.17,204.37,201.35,201.72,172.95,169632600,0,5,5,1,True,False,False,False,False,False
2015-01-06,202.09,202.72,198.86,199.82,171.32,209151400,1,6,6,1,False,False,False,False,False,False
2015-01-07,201.42,202.72,200.88,202.31,173.46,125346700,2,7,7,1,False,False,False,False,False,False
2015-01-08,204.01,206.16,203.99,205.90,176.54,147217800,3,8,8,1,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,435.96,438.28,435.54,438.11,438.11,67882300,3,29,180,6,False,False,False,False,False,False
2023-06-30,441.44,444.30,441.11,443.28,443.28,104921500,4,30,181,6,False,False,False,True,True,False
2023-07-03,442.92,444.08,442.63,443.79,443.79,32793400,0,3,184,7,True,True,False,False,False,False


Algorithm

In [5]:
pf.TradeLog.cash = capital

# Loop through timeseries.
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # On first trading day of the week, buy open, then sell close.
    if row.first_dotw:
        tlog.buy(date, row.open)
        tlog.sell(date, row.close)

    # Record daily balance.
    dbal.append(date, row.close, row.high, row.low)

Retrieve logs

In [6]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [7]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
439,2023-06-05,428.28,2023-06-05,427.10,-1.18,-29.50,25,753.27,LONG,SPY
440,2023-06-12,430.92,2023-06-12,433.80,2.88,69.12,24,822.39,LONG,SPY
441,2023-06-20,437.45,2023-06-20,437.18,-0.27,-6.48,24,815.91,LONG,SPY
442,2023-06-26,432.62,2023-06-26,431.44,-1.18,-29.50,25,786.41,LONG,SPY
443,2023-07-03,442.92,2023-07-03,443.79,0.87,20.88,24,807.29,LONG,SPY


Get stats

In [8]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [9]:
pf.summary(stats)

,strategy
annual_return_rate,0.92
max_closed_out_drawdown,-13.98
best_month,6.04
worst_month,-5.08
sharpe_ratio,0.20
sortino_ratio,0.12
monthly_std,1.41
annual_std,5.98
